In [10]:
import json
#from google.colab import drive

# Get the notebook's filename (usually matches the GitHub repo name)
!ls *.ipynb
notebook_name = "NLPProject.ipynb"  # ← Replace with your filename

# Load and fix the notebook
with open(notebook_name, 'r') as f:
    nb = json.load(f)

# Option A: Remove widgets metadata completely (recommended)
if 'metadata' in nb and 'widgets' in nb['metadata']:
    del nb['metadata']['widgets']

# Option B: Or add the missing state key
# if 'metadata' in nb and 'widgets' in nb['metadata']:
#     nb['metadata']['widgets']['state'] = {}

# Save the fixed version
with open(notebook_name, 'w') as f:
    json.dump(nb, f)

print("Notebook metadata fixed! You can now commit to GitHub.")

NLPProject.ipynb
Notebook metadata fixed! You can now commit to GitHub.


In [11]:
from pprint import pprint
from operator import itemgetter
import nltk
from nltk.corpus import framenet as fn
from nltk.corpus.reader.framenet import PrettyList
nltk.download('framenet_v17')


[nltk_data] Downloading package framenet_v17 to
[nltk_data]     /Users/kierstenwener/nltk_data...
[nltk_data]   Package framenet_v17 is already up-to-date!


True

Overview Notes

-FE is a frame element, the the frame evpking words are the lexical units (LU)
    -the concept of cooking involves the cook, the food that is being cooked, something holding the food while cooking, and the source of heat
    -this is represented by a frame called Apply_Heat and the Cook, Food, Heating_Instrument, and COntainer are the FEs; the words that evoke the frame (like fry, bake, boil, broil, etc) are the LUs

-inheritance: An IS-A relationship, the child frame is a subtype of the parent frame, each FE in the parent is bound to a corresponding FE in the child
    -ex; the "Revenge" frame which inherits from the "Rewards_and_punishments" frame
-using: The child frame presupposes the parent frame as background
    -eg: the "speed" frame "uses" the "Motion" frame, but not all parent FEs need to be bound to child FEs
-subframe: The child is a sibvent of a complex event represented by the parent
    -eg: the "Criminal_process" frame has subframes "arrest", "arraignment", "trial" and "sentencing"
-Perspective on: the child frame provices a particular perspective on an un-perspectivized parent frame; a pair of examples consists of "Hiring" and "Get_a_job" frames, which perspectivize the "Employment_start" frame from the Employers and Employees point of view

-use frames() to get list of all frames; can get particular frame and info by using frame(); can provide a regular expressions to frames() to get list of all frames whoe names match the patern
-can search Frames by their Lexical Units using frames_by_lemma(), which returns list of all frames that contain LUs in which the 'name' attrib of the LU matches the given regular expressiuon

In [12]:
f = fn.frame(202)
print(f.ID)
print(f.name)
print(f.definition)
print(f.frameRelations)

202
Arrest
Authorities charge a Suspect, who is under suspicion of having committed a crime (the Charges), and take him/her into custody.'The police arrested Harry on charges of manslaughter.'
[<Parent=Intentionally_affect -- Inheritance -> Child=Arrest>, <Complex=Criminal_process -- Subframe -> Component=Arrest>, ...]


Lexical Unit overview
- A lexical unit is a pairing of a word with a meaning
-the "Apply_Heat" frame describes a common situation involving a cook, some food, and a heating instrument and is_envoked_ by words such as bake, blanch, broil, brown, simmer, steak, etc
-The frame evoking words are the LUs in the Apply_head frame :
    -can be adjs, nouns, or verbs
    - reduction evokes "Cause_change_of_scalar_position" in .."the reduction of debt levels to 665 million from 2.6 billion"
-Framenet provides mult annotated examples of each sense of a word (i.e each LU)
-each LU is linked to a frame, and the other words that evoke that frame
    -framenet groups similar words together sematnically, like a thesaurous

In [13]:
#this is how you get details for a specific lexical unit
PrettyList(sorted(fn.lus(r'(?i)a little'), key=itemgetter('ID')))

[<lu ID=14733 name=a little.n>, <lu ID=14743 name=a little.adv>, ...]

-the LU named takes the form of the dotten string in which the lemme preceeds the"." and a part of speech follows the "."

In [14]:
print(fn.lu(256).name)
print(fn.lu(256).definition)
print(fn.lu(256).frame.name)
print(fn.lu(256).lexemes[0].name)

foresee.v
COD: be aware of beforehand; predict.
Expectation
foresee


get an annotated document with the docs() function

In [15]:
d = fn.docs('BellRinging')[0]
print(d.corpname)
print(d.sentence[49])

PropBank
full-text sentence (2824246) in BellRinging:


[POS] 17 tags

[POS_tagset] PENN

[text] + [annotationSet]

`` I live in hopes that the ringers themselves will be drawn into
             *****          *******                    *****     
             Desir          Cause_t                    Cause     
             [1]            [3]                        [2]       

 that fuller life .
      ******
      Comple
      [4]   
 (Desir=Desiring, Cause_t=Cause_to_make_noise, Cause=Cause_motion, Comple=Completeness)



Below, you get information about the first annotation in the specificed sentence above. We see that the word hope evokes the frame desiring

In [16]:
d.sentence[49].annotationSet[1]

annotation set (4528852):

[status] MANUAL

[LU] (6605) hope.n in Desiring

[frame] (366) Desiring

[GF] 2 relations

[PT] 2 phrases

[text] + [Target] + [FE] + [Noun]

`` I live in hopes that the ringers themselves will be drawn into
   - ^^^^ ^^ ***** ----------------------------------------------
   E supp su       Event                                         
 
 that fuller life .
-----------------
                 
 (E=Experiencer, su=supp)


find the training data :
-this is a whole set of the training data, can index it to look at specific frames

In [116]:
frames = fn.frames()
print(f"Number of frames: {len(frames)}")

Number of frames: 1221


-frame elements are the specific participants, roles, or components of a frame;
for each frame element,how many sample sentences do we have?

In [17]:
frame_element_counts = {}
#for each frame, loops through all frame elements
for frame in fn.frames():
    frame_name = frame.name

    for fe_name, fe in frame.FE.items():

        sample_sentences = frame.lexUnit
        num_sentences = len(sample_sentences)

        # Store the count of sentences for each frame element
        if fe_name in frame_element_counts:
            frame_element_counts[fe_name] += num_sentences  # Add the new count to the existing one
        else:
            frame_element_counts[fe_name] = num_sentences

sorted_frame_elements = sorted(frame_element_counts.items(), key=lambda x: x[1], reverse=True)
for fe_name, count in sorted_frame_elements[:10]:
    print(f"Frame Element: {fe_name}, Sample Sentences: {count}")

Frame Element: Time, Sample Sentences: 8170
Frame Element: Manner, Sample Sentences: 7612
Frame Element: Place, Sample Sentences: 7037
Frame Element: Degree, Sample Sentences: 7012
Frame Element: Means, Sample Sentences: 5045
Frame Element: Explanation, Sample Sentences: 4539
Frame Element: Depictive, Sample Sentences: 4091
Frame Element: Purpose, Sample Sentences: 4091
Frame Element: Circumstances, Sample Sentences: 3219
Frame Element: Duration, Sample Sentences: 3120


In [29]:
frames_with_time = []
for frame in fn.frames():
    for fe in frame['FE']:
        if fe == 'Time':
            frames_with_time.append(frame)

# Step 2: Show the names of frames that use the "Time" FE
print("Frames with Time FE:")
for f in frames_with_time:
    print(f["name"])

Frames with Time FE:
Abandonment
Abounding_with
Absorb_heat
Abundance
Abusing
Accomplishment
Accuracy
Achieving_first
Activity
Activity_abandoned_state
Activity_done_state
Activity_finish
Activity_ongoing
Activity_pause
Activity_paused_state
Activity_prepare
Activity_ready_state
Activity_resume
Activity_start
Activity_stop
Adding_up
Adducing
Adjacency
Adjusting
Adopt_selection
Aesthetics
Affirm_or_deny
Age
Aging
Agree_or_refuse_to_act
Agriculture
Aiming
Amalgamation
Amassing
Ambient_temperature
Annoyance
Appeal
Apply_heat
Appointing
Arraignment
Arranging
Arrest
Arriving
Arson
Artistic_style
Assemble
Assessing
Assistance
Atonement
Attaching
Attack
Attempt
Attempt_action_scenario
Attempt_distant_interaction_scenario
Attempt_means
Attempt_obtain_food_scenario
Attempt_obtain_mineral_scenario
Attempt_suasion
Attempting_and_resolving_scenario
Attending
Attitude_description
Avoiding
Awareness
Awareness_situation
Awareness_status
Bail_decision
Be_in_agreement_on_action
Be_in_agreement_on_asses

-we could have a target frame element, get all of the sentences associated with that frame element.
-First, we need to associate the sentence with a semantic frame, then parse to get the frame elements, or use bert to predict which word corresponds to a specific frame element

https://arxiv.org/html/2502.12516v1
-Json based formats show superior results, wrap target or word phrase in **


In [118]:
from transformers import pipeline

In [18]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())  # Should return True if you have a CUDA-compatible GPU


2.6.0
False


### TAGGING

Started tagging using the listed top ten words. Added a way for context to be acknowledged while attempting to tag.

In [19]:


import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import framenet as fn
nltk.download('averaged_perceptron_tagger_eng')
import re
from nltk import pos_tag
from transformers import BertTokenizer, BertModel

#  pre-trained BERT model and tokenizer ( this will provide context for example sentences)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

top_10_fes = [
    "Time", "Manner", "Place", "Degree", "Means", "Explanation",
    "Depictive", "Purpose", "Circumstances", "Duration"
]

# dictionary to store lexical units for the top 10 frame elements
frame_element_lexemes = {}

# function to strip POS ( parts-of-speech) tags from lexemes
def strip_pos_tag(lexeme):
    return re.sub(r'\.[a-z]+$', '', lexeme)

# loop through all frames in FrameNet to gather lexical units for each of the top 10 frame elements
for frame in fn.frames():
    frame_name = frame.name  # Frame name (e.g., "Cause", "Motion")

    # loop through each frame element in the frame
    for fe_name, fe in frame.FE.items():
        if fe_name in top_10_fes:
            # get all lexical units for this frame element
            if fe_name not in frame_element_lexemes:
                frame_element_lexemes[fe_name] = set()
            # if it's a lexical unit object, access its lexemes
            for lu in frame.lexUnit:
                if hasattr(lu, 'lexemes'):
                    for lexeme in lu.lexemes:
                        lexeme_name = strip_pos_tag(lexeme.name.lower())  # remove POS tag
                        frame_element_lexemes[fe_name].add(lexeme_name)  # store lexemes in lowercase without POS tag
                else:
                    # if it's just a string, we directly add it as a lexeme
                    lexeme_name = strip_pos_tag(lu.lower())  # remove POS tag
                    frame_element_lexemes[fe_name].add(lexeme_name)

# function to get BERT embeddings for a sentence
def get_bert_embeddings(sentence):
    # tokenize and encode sentence
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)

    # get the embeddings from BERT
    with torch.no_grad():
        outputs = bert_model(**inputs)

    # return the token embeddings
    return outputs.last_hidden_state

# function to tag a sentence with context
def tag_sentence_with_context(sentence):
    tokens = word_tokenize(sentence)
    token_embeddings = get_bert_embeddings(sentence)

    tagged_sentence = []
    pos_tags = pos_tag(tokens)  # POS tagging to help with filtering

    # loop through each word in the sentence
    for i, (word, pos) in enumerate(pos_tags):
        word_lower = strip_pos_tag(word.lower())  # Convert word to lowercase and strip POS tag
        assigned_fe = "none"

        # check each top frame element for the word
        for fe_name, lexemes in frame_element_lexemes.items():
            if word_lower in lexemes:
                # use BERT embeddings and POS tags to determine if the word is related to the correct FE
                embedding = token_embeddings[0, i].cpu().numpy()

                # apply conditions to check if the word is related to the correct FE based on its embedding and POS tag
                if fe_name == "Time" and pos in ['NN', 'NNS', 'VB', 'VBP']:  # time is often linked to nouns/verbs
                    assigned_fe = fe_name
                    break
                elif fe_name == "Place" and pos in ['NN', 'NNS']:  # place-related words are usually nouns
                    assigned_fe = fe_name
                    break
                elif fe_name == "Degree" and pos in ['RB', 'JJ']:  # degree-related words are often adverbs or adjectives
                    assigned_fe = fe_name
                    break
                elif fe_name == "Manner" and pos in ['RB']:  # manner-related words are adverbs
                    assigned_fe = fe_name
                    break
                elif fe_name == "Means" and pos in ['NN', 'NNS', 'IN']:  # means-related words are nouns or prepositions
                    assigned_fe = fe_name
                    break
                elif fe_name == "Purpose" and pos in ['NN', 'VB', 'IN']:  # purpose-related words are nouns, verbs, or prepositions
                    assigned_fe = fe_name
                    break
                elif fe_name == "Explanation" and pos in ['NN', 'VB']:  # explanation-related words are nouns or verbs
                    assigned_fe = fe_name
                    break
                elif fe_name == "Circumstances" and pos in ['NN', 'IN', 'JJ', 'RB']:  # circumstances-related words are nouns, prepositions, adjectives, or adverbs
                    assigned_fe = fe_name
                    break
                elif fe_name == "Depictive" and pos in ['JJ', 'RB']:  # depictive-related words are adjectives or adverbs
                    assigned_fe = fe_name
                    break
                elif fe_name == "Duration" and pos in ['NN', 'VB', 'JJ', 'RB']:  # duration-related words are nouns, verbs, adjectives, or adverbs
                    assigned_fe = fe_name
                    break

        tagged_sentence.append((word, assigned_fe))

    return tagged_sentence

# Example sentences for testing
sentences = [
    "The meeting will begin at 3 PM.",
    "She worked all night to finish the project.",
    "The concert starts tomorrow.",
    "She explained the concept clearly.",
    "The team worked together efficiently.",
    "He ran away nervously.",
    "They visited the park in the morning.",
    "I left my keys on the kitchen counter.",
    "The store is near the bus station.",
    "It was so hot that I couldn’t stand outside.",
    "He’s extremely talented.",
    "The cake was quite delicious.",
    "She achieved her goals through hard work.",
    "We traveled by car across the country.",
    "The results were obtained using advanced techniques.",
    "He explained why he was late for the meeting.",
    "The teacher clarified the rules of the game.",
    "She gave a detailed explanation about the new policy.",
    "She looked tired after the long trip.",
    "The sky was clear and blue.",
    "The dog seemed excited and playful.",
    "He went to the store to buy groceries.",
    "The project was created to improve customer service.",
    "She studies hard to become a doctor.",
    "Despite the rain, they went for a walk.",
    "Under normal circumstances, the event would be postponed.",
    "In the event of a fire, use the emergency exit.",
    "He lived in Paris for two years.",
    "The lecture lasted for three hours.",
    "She was absent for a week due to illness."
]

# process each sentence and tag it
tagged_sentences = []

for sentence in sentences:
    tagged = tag_sentence_with_context(sentence)
    tagged_sentences.append(tagged)

# print and test sentences
for sentence, tagged in zip(sentences, tagged_sentences):
    print(f"Original Sentence: {sentence}")
    print(f"Tagged Sentence: {tagged}")
    print("-" * 50)




[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/kierstenwener/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
/Users/kierstenwener/Desktop/NLPProject/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Original Sentence: The meeting will begin at 3 PM.
Tagged Sentence: [('The', 'none'), ('meeting', 'Place'), ('will', 'none'), ('begin', 'Time'), ('at', 'Purpose'), ('3', 'none'), ('PM', 'none'), ('.', 'none')]
--------------------------------------------------
Original Sentence: She worked all night to finish the project.
Tagged Sentence: [('She', 'none'), ('worked', 'none'), ('all', 'none'), ('night', 'none'), ('to', 'none'), ('finish', 'Time'), ('the', 'none'), ('project', 'Place'), ('.', 'none')]
--------------------------------------------------
Original Sentence: The concert starts tomorrow.
Tagged Sentence: [('The', 'none'), ('concert', 'none'), ('starts', 'none'), ('tomorrow', 'none'), ('.', 'none')]
--------------------------------------------------
Original Sentence: She explained the concept clearly.
Tagged Sentence: [('She', 'none'), ('explained', 'none'), ('the', 'none'), ('concept', 'none'), ('clearly', 'Degree'), ('.', 'none')]
--------------------------------------------

It future, could compare the cosine similarity of the bert embeddings and the FE and choose the one with the highest cosine similarity: 
get average bert embeddings for all of the top 10, for a new word in a sentence, compare its embedding to the average FE embeddings and 

Prepare Training Data:
Use your frame_element_lexemes dictionary to label known tokens in example sentences with their correct FE (this will serve as weak supervision).
For each labeled token:
    Extract its BERT embedding.
    Get its POS tag.
    Store (embedding, POS tag, FE label) as one training example.
    Encode POS Tags (e.g., one-hot or label encoding).

Train a Classifier:
    You can use sklearn's LogisticRegression, RandomForestClassifier, or even a small neural net.
    Input: [BERT embedding + POS encoding]
    Output: FE label (Time, Place, etc.)
    
Predict on New Sentences:
For each token:
    Get BERT embedding + POS.
    Feed to classifier to predict the FE.

predicting which word plays a specific role. 

- encode sentence with bert 
-encode the frame element, like fixed label embedding

for training set we need the sentence, the role, and the index of the word in the sentence that has the role 

TRAINING AND TESTING ENCODER DECODER

In [94]:
def align_labels_with_tokens(text, char_labels, offsets):
    token_labels = []
    for (token_start, token_end) in offsets:
        # For special tokens like [CLS] and [SEP], offset is usually (0,0)
        if token_start == token_end:
            token_labels.append("O")
        else:
            # If any character in the token is marked as Time,
            # decide on a label for the entire token.
            token_tag = "O"
            for pos in range(token_start, token_end):
                if pos < len(char_labels) and char_labels[pos] != "O":
                    token_tag = char_labels[pos]
                    break
            token_labels.append(token_tag)
    return token_labels


i need to find all LUs that have the frame element time, then i can find all sentences for that LU

In [128]:
frames_with_time_ex = {}
for f in fn.frames():
    for x in f.FE:
        if x == "Time":
            frames_with_time_ex[f.name] = f


In [140]:
tagged_examples = []
    
for name, frame in frames_with_time_ex.items():
    # Print the frame name for reference
    for lu in frame.lexUnit.values():
        print(f"\nLexical Unit: {lu['name']}")
        lu_data = fn.lu(lu['ID'])
        for ex in lu_data['exemplars']:
            print("  Example sentence:")
            print("   ", ex['text'])
            for ann in ex['annotationSet']:
                if 'frame' in ann:
                    print("  Frame Elements:")
                    for layer in ann['layer']:
                        if layer['name'] == 'FE':  # This layer contains frame elements
                            for label in layer['label']:
                                start = label['start']
                                end = label['end']
                                fe_name = label['name']
                                fe_text = " ".join(ex['token'][start:end])
                                print(f"    {fe_name}: {fe_text}")





Lexical Unit: abandon.v
  Example sentence:
    The bank has abandoned all plans to finance roads or logging in Cameroon 's forests , in keeping with its ` stringent policy to protect the rights of indigenous people . "
  Frame Elements:
  Frame Elements:


KeyError: 'token'

In [106]:
import torch
from nltk.corpus import framenet as fn
from transformers import BertTokenizerFast

# Initialize tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Map BIO tags to IDs
label2id = {"O": 0, "B-Time": 1, "I-Time": 2}

input_ids_list = []
attention_masks_list = []
labels_list = []

# Find frames that include "Time" as a frame element
frames_with_time = [f for f in fn.frames() if 'Time' in f.FE]

for frame in frames_with_time:
    for lexunit_key in frame['lexUnit']:
        lexunit = frame['lexUnit'][lexunit_key]
        for ex in lexunit.get('exemplars', []):
            text = ex['text']
            char_labels = ["O"] * len(text)

            # Go through annotation layers
            for anno in ex['annotationSet']:
                for layer in anno.get('layers', []):
                    if layer['name'] == 'FE':
                        for item in layer.get('labels', []):
                            if item['name'] == 'Time':
                                print("time element found:")
                                start, end = item['start'], item['end']
                                print(f"{start}, {end}")
                                if start < end:
                                    char_labels[start] = "B-Time"
                                    for i in range(start+1, end):
                                        char_labels[i] = "I-Time"

            # Tokenize
            tokenized = tokenizer(text, return_offsets_mapping=True, truncation=True, padding="max_length", max_length=128)
            input_ids = tokenized["input_ids"]
            attention_mask = tokenized["attention_mask"]
            offsets = tokenized["offset_mapping"]

            # Map character-level labels to token-level labels
            token_labels = align_labels_with_tokens(text, char_labels, offsets)
            label2id_binary = {"O": 0, "B-Time": 1, "I-Time": 1}  # Map both B-Time and I-Time to 1
            label_ids = [label2id_binary.get(lab, 0) for lab in token_labels]

            # Store tensors
            input_ids_list.append(torch.tensor(input_ids))
            attention_masks_list.append(torch.tensor(attention_mask))
            labels_list.append(torch.tensor(label_ids))

# Final dataset tensors
input_ids_tensor = torch.stack(input_ids_list)
attention_masks_tensor = torch.stack(attention_masks_list)
labels_tensor = torch.stack(labels_list)

print("Tensor shapes:")
print("Input IDs:", input_ids_tensor.shape)
print("Attention Masks:", attention_masks_tensor.shape)
print("Labels:", labels_tensor.shape)

TypeError: 'NoneType' object is not iterable

In [96]:
from torch.utils.data import TensorDataset

dataset = TensorDataset(input_ids_tensor, attention_masks_tensor, labels_tensor)


In [97]:
from torch.utils.data import DataLoader, RandomSampler

# Parameters
batch_size = 16

# Create a DataLoader
train_dataloader = DataLoader(
    dataset,
    sampler=RandomSampler(dataset),  # shuffle the data
    batch_size=batch_size
)


In [98]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel

class FrameElementClassifier(nn.Module):
    def __init__(self, bert_model='bert-base-uncased'):
        super().__init__()
        self.bert = BertModel.from_pretrained(bert_model)
        self.query_encoder = nn.Linear(self.bert.config.hidden_size, self.bert.config.hidden_size)
        self.token_projection = nn.Linear(self.bert.config.hidden_size, self.bert.config.hidden_size)
        self.classifier = nn.Linear(self.bert.config.hidden_size, 2)
        self.dropout = nn.Dropout(0.1)

    #def forward(self, input_ids, attention_mask, role_ids, role_mask):
    def forward(self, input_ids, attention_mask):
        # Encode sentence
        sentence_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        token_embeddings = sentence_outputs.last_hidden_state  # shape: (B, T, H)
        """
        # Encode role label (like "Time" or "Manner")
        role_output = self.bert(input_ids=role_ids, attention_mask=role_mask)
        role_embedding = role_output.last_hidden_state[:, 0, :]  # [CLS] token: shape (B, H)
        role_embedding = self.query_encoder(role_embedding)  # shape: (B, H)
"""
        # Project sentence tokens
        token_embeddings = self.token_projection(token_embeddings)  # shape: (B, T, H)
        logits = self.classifier(token_embeddings)
        # Compute dot product between role embedding and each token
        #role_embedding = role_embedding.unsqueeze(2)  # (B, H, 1)
        #scores = torch.bmm(token_embeddings, role_embedding).squeeze(-1)  # shape: (B, T)

        # Optionally apply attention mask
        #scores = scores.masked_fill(attention_mask == 0, -1e9)
        #probs = torch.softmax(logits, dim=-1)
        logits = self.classifier(token_embeddings)
        return logits  # Apply softmax for inference or use with CrossEntropyLoss

In [99]:
print(len(train_dataloader))

7749


In [100]:
import torch
model = FrameElementClassifier()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
# Automatically choose GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
num_batches = 15
print("Using device:", device)
for i, batch in enumerate(train_dataloader):
    if i >= num_batches:
        break
    input_ids, attention_mask, target_index = [item.to(device) for item in batch]

    # Zero the gradients before the backward pass
    optimizer.zero_grad()

    # Forward pass
    probs = model(input_ids, attention_mask)

    # Calculate the loss (for each token, it should either be "Time" (1) or "Non-Time" (0))
    loss = loss_fn(probs.view(-1, 2), target_index.view(-1))  # Flatten and match shape for loss

    # Backward pass
    loss.backward()

    # Update parameters
    optimizer.step()

    # Optional: print loss or other metrics for monitoring
    print(f"Loss: {loss.item()}")


Using device: cpu
Loss: 0.647087812423706
Loss: 0.5493744015693665
Loss: 0.4674415588378906
Loss: 0.3933328092098236
Loss: 0.31572821736335754
Loss: 0.2356415092945099
Loss: 0.1841731071472168
Loss: 0.12174943089485168
Loss: 0.07833168655633926
Loss: 0.04149963706731796
Loss: 0.028079401701688766
Loss: 0.018292853608727455
Loss: 0.01723303832113743


KeyboardInterrupt: 

In [101]:
# Initialize the model
model = FrameElementClassifier()

# Set up device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
num_epochs = 3
# Define loss function and optimizer
loss_fn = nn.CrossEntropyLoss()  # Assuming target_index is 0 for "Non-Time" or 1 for "Time"
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# Number of batches to train on
num_batches = 5

# To track accuracy
accuracies = []

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_predictions_batch = 0
    total_predictions = 0
    
    for i, batch in enumerate(train_dataloader):
        if i >= num_batches:
            break
        
        input_ids, attention_mask, target_index = [item.to(device) for item in batch]
        print("Unique target values:", torch.unique(target_index))
        
        # Zero the gradients before the backward pass
        optimizer.zero_grad()

        # Forward pass
        probs = model(input_ids, attention_mask)

        # Calculate the loss
        loss = loss_fn(probs.view(-1, 2), target_index.view(-1))
        
        # Backward pass
        loss.backward()

        # Update parameters
        optimizer.step()

        # Track running loss
        running_loss += loss.item()

    with torch.no_grad():
        probs_softmax = torch.softmax(probs, dim=-1)
        time_probs = probs_softmax[..., 1]  # (B, T)
        predicted_time_token = torch.argmax(time_probs, dim=-1)  # (B,)

        true_time_token = (target_index == 1).nonzero(as_tuple=False)  # (#T, 2)
        true_time_map = {int(b): int(t) for b, t in true_time_token}

        correct_predictions = sum(
            predicted_time_token[b] == true_time_map.get(b, -1)
            for b in range(input_ids.size(0))
        )

        total_predictions = input_ids.size(0)
        correct_predictions_batch += correct_predictions

    epoch_accuracy = correct_predictions_batch / total_predictions
    print(epoch_accuracy)




Unique target values: tensor([0])
Unique target values: tensor([0])
Unique target values: tensor([0])
Unique target values: tensor([0])
Unique target values: tensor([0])
tensor(0.)
Unique target values: tensor([0])
Unique target values: tensor([0])
Unique target values: tensor([0])
Unique target values: tensor([0])
Unique target values: tensor([0])
tensor(0.)
Unique target values: tensor([0])
Unique target values: tensor([0])
Unique target values: tensor([0])
Unique target values: tensor([0])
Unique target values: tensor([0])
tensor(0.)


In [ ]:
import matplotlib.pyplot as plt

plt.plot(accuracies)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Model Accuracy Over Time')
plt.show()